Exercice initialement proposé par Élodie Puybareau, remis en forme par Guillaume Tochon

In [ ]:
### Import des bibliothèques ###
import numpy as np
import scipy as sp
from scipy import signal
import skimage
from skimage import io
import matplotlib.pyplot as plt
# Matplotlib en mode inline
%matplotlib inline

# Le produit de convolution (en traitement d'images)

Dans cet exercice, vous allez mettre en œuvre le produit de convolution pour des problématiques de filtrage. Plus particulièrement, vous allez travailler sur des signaux un peu particuliers, à savoir des images, c'est-à-dire des signaux en deux dimensions.

## Du signal à l'image

Commençons par rappeler la définition du produit de convolution pour deux signaux $x : \mathbb{R} \rightarrow \mathbb{R}$ et $y : \mathbb{R} \rightarrow \mathbb{R}$ continus (par opposition à discret, comme c'est le cas d'une image) :

$$(x \ast y)(\tau) = \int_\mathbb{R} x(t) y (\tau - t) dt \qquad (1)$$

L'extension du produit de convolution à des signaux bidimensionnels $x : \mathbb{R}^2 \rightarrow \mathbb{R}$ et $y : \mathbb{R}^2 \rightarrow \mathbb{R}$ se fait naturellement en rajoutant une variable supplémentaire à $x$ et $y$ (appelons la $u$), et une variable supplémentaire également au produit de convolution $x \ast y$ (appelons la $\nu$):

$$ (x \ast y)(\tau, \nu) = \iint_{\mathbb{R}^2} x(t,u) y (\tau - t, \nu - u) dt du \qquad (2)$$

Une image étant un signal 2D discret (c'est-à-dire une matrice $X[i,j], i=0,\dots,M-1, j=0,\dots,N-1$ avec $M$ et $N$ le nombre de lignes et de colonnes), on peut adapter "facilement" la formule $(2)$ pour des signaux de cette nature : si $X$ et $Y$ sont deux images (supposées de taille infinie dans un premier temps), alors

$$(X \ast Y)[m,n] = \sum_{i=-\infty}^{+\infty} \sum_{j=-\infty}^{+\infty} X[i,j]Y[m-i,n-j] \qquad (3)$$

En pratique, lorsqu'on fait du filtrage en traitement d'image, on calcule la convolution d'une image $X$ par un [<b>noyau</b>](https://fr.wikipedia.org/wiki/Noyau_(traitement_d%27image)) $K$ (également appelé <b>filtre</b>, ou <i>kernel</i> en anglais) d'une taille $(2P+1) \times (2P+1)$ plus petite (typiquement $3\times 3$ ou $5 \times 5$). L'équation (3) peut se réécrire alors en

$$ Y[m,n] = (X \ast K)[m,n] = \sum_{i=m-P}^{m+P} \sum_{j=n-P}^{n+P} X[i,j]K[m-i,n-j] = \sum_{i=-P}^P \sum_{j=-P}^P K[i,j]X[m-i,n-j] \qquad (4)$$

À noter qu'un noyau de convolution est toujours de taille impaire ($3 \times 3$, $5 \times 5$, $9 \times 9$, etc) afin qu'il admette un pixel central.

La convolution en 1D [s'illustre](https://en.wikipedia.org/wiki/Convolution#/media/File:Convolution_of_spiky_function_with_box2.gif) par un signal fixe et un signal glissant. La même illustration existe dans le cas 2D pour la convolution d'une image par un noyau : dans ce cas là, l'image correspond au signal fixe, et le noyau "glisse" sur toutes les positions de l'image. Pour un pixel donné dans l'image, on fait la somme des produits termes à termes des valeurs de l'image couvertes par le noyau avec leur coefficient respectif du noyau. Le résultat donne la valeur du pixel après convolution. 

## La convolution pour une image

## La convolution pour une image

![Alt Text](https://miro.medium.com/max/1400/1*O06nY1U7zoP4vE5AZEnxKA.gif)

L'animation ci-dessus illustre le mécanisme de la convolution en 2D :

* <span style="color: royalblue;">L'image $X$</span>, représentée comme une grille de valeurs en 2 dimensions. Chaque case correspond à la position d'un pixel, et chaque valeur associée représente le niveau de gris du pixel avec $0$ représentant le noir ⬛, 255 représentant le blanc ⬜ (lorsque les pixels sont encodés sur 8 bits), et chaque valeur intermédiaire représentant une nuance de gris plus ou moins foncée.
* <span style="color: darkorange;">Le noyau de convolution $K$</span>. Contrairement à l'image, ses valeurs (appelées <b>poids</b>) peuvent être positives ou négatives.
* <span style="color: forestgreen;">Le résultat de la convolution $Y = X \ast K$</span>, que l'on appelle aussi <b>image filtrée</b>. Ses valeurs peuvent également être positives ou négatives (en fonction des poids du filtre $K$), et doivent donc être interprétées avec précaution, puisqu'elles ne représentent plus nécéssairement des niveaux de gris.

Dans cet exemple, le résultat a la même taille que l'image initiale $X$, ce qui veut dire que pour chaque pixel $X[i,j]$ de l'image initiale, une nouvelle valeur est calculée. Cette nouvelle valeur correspond à la somme de la multiplication terme à terme de deux matrices : celle du noyau et une sous-matrice de la même taille que le noyau, centrée sur le pixel $X[i,j]$ étudié.

Prenons le pixel central de l'exemple ci-dessus : le pixel "70". En centrant le noyau dessus, on va avoir :

<font color="royalblue"> 121</font>x<font color="darkorange">0</font>+<font color="royalblue">54</font>x<font color="darkorange">(-1)</font>+<font color="royalblue"> 84</font>x<font color="darkorange">0</font>+<font color="royalblue"> 99</font>x<font color="darkorange">(-1)</font>+<font color="royalblue">70</font>x<font color="darkorange">5</font>+<font color="royalblue"> 129</font>x<font color="darkorange">(-1)</font>+<font color="royalblue"> 57</font>x<font color="darkorange">0</font>+<font color="royalblue">115</font>x<font color="darkorange">(-1)</font>+<font color="royalblue"> 69</font>x<font color="darkorange">0</font> = <font color="forestgreen">-47</font>

#### Que se passe-t-il aux bords ? 
La bordure blanche autour de l'image bleue représente le <i>padding</i>. Il s'agit d'un ajout que l'on fait aux bordures de l'image pour que le noyau ne déborde pas de l'image au moment de la convolution. En effet, le noyau centré sur un pixel du bord va déborder et retourner une erreur. Pour éviter cela, on rajoute artificiellement autant de lignes et de colonnes que de débordement : c'est le <i>padding</i>. Ici, ce padding est fait avec des 0, mais il peut également être fait avec la valeur moyenne de l'image, avec une duplication miroir des lignes et des colonnes etc. Si l'on reprend le calcul à la main sur le premier pixel de l'image, le "60", on a donc : 

<font color="royalblue"> 0</font>x<font color="darkorange">0</font>+<font color="royalblue">0</font>x<font color="darkorange">(-1)</font>+<font color="royalblue"> 0</font>x<font color="darkorange">0</font>+<font color="royalblue"> 0</font>x<font color="darkorange">(-1)</font>+<font color="royalblue">60</font>x<font color="darkorange">5</font>+<font color="royalblue"> 113</font>x<font color="darkorange">(-1)</font>+<font color="royalblue"> 0</font>x<font color="darkorange">0</font>+<font color="royalblue">73</font>x<font color="darkorange">(-1)</font>+<font color="royalblue"> 121</font>x<font color="darkorange">0</font> = <font color="forestgreen">114</font>

### 🛠️ 🚧 👷  À vous de jouer !

Quelle est la taille du padding (c'est-à-dire, le nombre de lignes à rajouter en haut et en bas de l'image, et le nombre de colonnes à rajouter à gauche et à droite de l'image) si le noyau de convolution $K$ est de taille $(2P+1)\times(2P+1)$ ?

Si le noyau de convolution $K$ est de taille $(2P+1)\times(2P+1)$, alors la taille du padding est de FIXME lignes/colonnes de chaque côté de l'image.

## La convolution comme opération de filtrage

Il est maintenant temps de voir la convolution en action pour des images ! Première étape : chargeons une image.

In [ ]:
# 🛑 Chargement et affichage d'une image
img = skimage.io.imread('image_chat.jpg')
img = np.uint8(skimage.color.rgb2gray(img)*255) # conversion de l'image couleur en niveaux de gris
# Affichage de l'image
plt.figure(figsize=(8,5))
plt.imshow(img,cmap='gray')
plt.show()

In [ ]:
# 🛑 Affichage d'informations basiques sur l'image
print("L'image possède",img.shape[0],"lignes et",img.shape[1],"colonnes")
print("La valeur min de l'image est",img.min(),"et la valeur max",img.max())
print("Le type d'encodage est",img.dtype)

### Une sombre histoire de type

Par défaut, les images sont encodées en `uint8`, ce qui veut dire que seules les valeurs entières entre $0$ et $255$ sont encodables. Or, les poids du noyau de convolution peuvent être arbitraires (donc soit négatifs, soit supérieurs à $255$), et les valeurs de l'image après convolution également (cf illustration ci-dessus).

Avant toute chose, il convient donc de caster l'encodage de l'image vers un type nettement moins propice aux débordements de valeurs, à savoir le `float`.

In [ ]:
# 🛑 cast l'image en type float
img = img.astype(float)
print("La valeur min de l'image est %1.1f, et la valeur max %1.1f"%(img.min(),img.max()))
print("Le type d'encodage est",img.dtype)

## Définition d'un premier noyau

Définissons un premier noyau de convolution : $K_1 = \displaystyle\frac{1}{9}\begin{bmatrix}1&1&1\\1&1&1\\1&1&1\end{bmatrix}$.

### 🛠️ 🚧 👷  À vous de jouer !

Quelle opération permet de réaliser $K_1$ lorsqu'on le convolue à une image $X$ ? Plus particulièrement, quel est le résultat (littéral) de la convolution du pixel $X[i,j]$ (en fonction de ses voisins) par $K_1$ ?

$$
\begin{bmatrix} \cdot & \cdot & \cdot & \cdot & \cdot \\
\cdot & X[i-1,j-1] & X[i-1,j] & X[i-1,j+1] & \cdot \\
\cdot & X[i,j-1] & X[i,j] & X[i,j+1] & \cdot \\
\cdot & X[i+1,j-1] & X[i+1,j] & X[i+1,j+1] & \cdot \\
\cdot & \cdot & \cdot & \cdot & \cdot
\end{bmatrix} \ast \displaystyle\frac{1}{9}\begin{bmatrix}1&1&1\\1&1&1\\1&1&1\end{bmatrix}$$

Le résultat de la convolution est 

$$\displaystyle\frac{?X[i-1,j-1]+?X[i-1,j]+?X[i-1,j+1]+?X[i,j-1]+?X[i,j]+?X[i,j+1]+?X[i+1,j-1]+?X[i+1,j]+?X[i+1,j+1]}{???}$$

Le noyau $K_1$ permet donc de réaliser une opération de FIXME.

### 🛠️ 🚧 👷  À vous de jouer !

Il est maintenant temps de mettre ce filtre à l'œuvre. Pour implémenter l'opération de convolution, on va utiliser [`sp.signal.convolve2d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html#scipy.signal.convolve2d), optimisée pour la convolution d'images. À noter qu'il en existe une version générique, à savoir [`sp.signal.convolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html) qui fonctionne pour des signaux à N dimensions. Mais puisque nous allons travailler ici exclusivement avec des images, on lui préférera donc `sp.signal.convolve2d`.

Calculez la convolution de l'image chargée précédemment par le noyau $K_1$ (dans un premier temps, ne vous souciez pas des options <i>mode</i> et <i>boundary</i> de `sp.signal.convolve2d`).<br> Quelle est la taille (nombre de lignes et nombre de colonnes) du résultat de la convolution ? Comment expliquer la différence de taille par rapport à celle de l'image originale ?

In [ ]:
# 🛑 Définition du noyau K1
K1 = np.ones((3,3))
K1 = K1/K1.sum()
print("Noyau de convolution K1 :")
print(K1)

In [ ]:
img_conv = ??? # FIXME

In [ ]:
print("La taille de l'image après convolution est",???) # FIXME

La différence de taille entre l'image avant convolution et l'image après convolution vient de FIXME.

Lorsque l'on fait du traitement d'images, on n'aime en général pas trop les traitements qui <b>changent</b> la taille de l'image (sauf si c'est fait de manière intentionnelle, comme par exemple un zoom...). Pour pallier au problème précédent, il faut régler l'option `mode='same'` dans `sp.signal.convolve2d`.

### 🛠️ 🚧 👷  À vous de jouer !

Vérifiez que l'ajout de l'option `mode='same'` dans `sp.signal.convolve2d` permet bien d'avoir une image après convolution de même taille que l'image originale.

In [ ]:
img_conv = ??? # FIXME

Pour ne pas avoir à écrire à chaque fois `sp.signal.convolve2d(xx,yy,mode='same')`, on va définir le raccourci suivant :

In [ ]:
# 🛑 Raccourci
def conv2d(in1,in2):
    return sp.signal.convolve2d(in1,in2,mode='same')

print((conv2d(img,K1) == sp.signal.convolve2d(img,K1,mode='same')).all())

Calculer des convolutions quand on fait du traitement d'images, c'est bien. Mais les visualiser pour en apprécier les effets, c'est mieux !

In [ ]:
# 🛑 Affichage des résultats de la convolution + zoom
plt.figure(figsize=(15,8))
plt.subplot(1,2,1)
plt.imshow(img,cmap='gray',vmin=0,vmax=255)
plt.subplot(1,2,2)
plt.imshow(img_conv,cmap='gray',vmin=0,vmax=255)
plt.show()
# zoom
plt.figure(figsize=(15,8))
plt.subplot(1,2,1)
plt.imshow(img[300:400,200:300],cmap='gray',vmin=0,vmax=255)
plt.subplot(1,2,2)
plt.imshow(img_conv[300:400,200:300],cmap='gray',vmin=0,vmax=255)
plt.show()

### 🛠️ 🚧 👷  À vous de jouer !

Quel effet observez-vous ?

On observe un effet de FIXME.

## Effet de la taille du noyau sur le résultat de la convolution

Le noyau $K_1$ précédemment défini était de taille $3 \times 3$ : pour chaque pixel, il agit donc dans un voisinage immédiat, en ne considérant que les voisins directs du pixel en question. Évidemment, un noyau de convolution peut être plus gros que $3 \times 3$, du moment que sa taille reste impaire. Regardons maintenant l'impact de cette taille de noyau sur le résultat de la convolution, en définissant tout d'abord une fonction permettant de générer un noyau de taille $(2P+1)\times(2P+1)$ de la forme
$$K_p = \displaystyle\frac{1}{(2P+1)^2} \begin{bmatrix}1 & \dots & 1 \\ \vdots & \ddots & \vdots \\ 1 & \cdots & 1 \end{bmatrix} $$

In [ ]:
# 🛑 Définition d'un noyau moyenneur de taille arbitraire
def Kp(p):
    Kp = np.ones((2*p+1,2*p+1))
    return Kp/Kp.sum()

print((Kp(1)==K1).all())

### 🛠️ 🚧 👷  À vous de jouer !

Calculez et visualisez la convolution de l'image originale par des noyaux de différentes tailles : $5 \times 5$, $9 \times 9$, $15 \times 15$, etc.<br>Que pouvez-vous en conclure à propos de l'influence de la taille du filtre sur le résultat de la convolution ?

In [ ]:
??? # FIXME

L'influence de la taille du filtre sur le résultat de la convolution est FIXME.

## Définition d'un deuxième noyau

Les noyaux définis dans la partie précédente ont comme effet de lisser l'image après convolution. Dans le jargon, on dit que ce sont des filtres "passe-bas" : ils atténuent les hautes fréquences de l'image (à savoir ses variations rapides), ne laissant que les basses fréquences (les variations lentes, donc floues).

À l'inverse, un filtre "passe-haut" coupe les basses fréquences, ne laissant donc que les hautes fréquences (les variations rapides, donc les contours).
Un filtre passe-haut classique est le filtre de [Prewitt](https://en.wikipedia.org/wiki/Prewitt_operator) : \
$P_x = \begin{bmatrix} 1 & 0 & -1\\1 & 0 & -1\\1 & 0 & -1 \end{bmatrix} \qquad P_y = \begin{bmatrix} 1 & 1 & 1\\0 & 0 & 0\\-1 & -1 & -1 \end{bmatrix}$

$P_x$ permet de détecter les contours horizontaux, alors que $P_y$ permet de détecter les contours verticaux.

<i>Note : si vous voulez une justification de la définition de ce filtre (et bien d'autres), l'électif IMAGIN au S6 sera là pour ça 🙂 (uniquement sur Paris, Lyon et Strasbourg).</i>

### Convolution d'une image par $P_x$


### 🛠️ 🚧 👷  À vous de jouer !

Définissez le noyau de convolution $P_x$, appliquez le à l'image originale précédente, et affichez le résultat. Que remarquez-vous ?

In [ ]:
Px = np.array([[???],[???],[???]]) # FIXME
img_conv_px = ??? # FIXME
??? # FIXME

On remarque que FIXME.

### Convolution d'une image par $P_Y$


### 🛠️ 🚧 👷  À vous de jouer !

Mêmes questions que précédemment, mais pour le noyau $P_y$ cette fois.

In [ ]:
Py = np.array([[???],[???],[???]]) # FIXME
img_conv_py = ??? # FIXME
??? # FIXME

On remarque que FIXME.

## Détection des contours par convolution

Les images obtenues après convolution par $P_x$ et $P_y$ sont appelées <i>gradients directionnels</i> : $\nabla_x X = X \ast P_x$ et $\nabla_y X = X \ast P_y$, et permettent de détecter les contours horizontaux et verticaux séparément.

Il est possible de combiner ces deux gradients directionnels pour obtenir :
* l'intensité du gradient pour chaque pixel, donnée grâce à sa norme $\| \nabla X\| = \displaystyle\sqrt{(\nabla_x X)^2+(\nabla_y X)^2}$. Il est ensuite possible de seuiller $\| \nabla X\|$ de manière judicieuse pour obtenir une image binaire des contours.
* l'orientation du gradient grâce à $\theta(x) = \arctan\left(\displaystyle\frac{\nabla_y X}{\nabla_x X}\right)$

In [ ]:
# 🛑 Définition et calcul de l'intensité et de l'orientation du gradient
def grad_intensity(nabla_x_im,nabla_y_im):
    return np.sqrt(nabla_x_im**2+nabla_y_im**2)

img_grad = grad_intensity(img_conv_px,img_conv_py)

def grad_direction(nabla_x_im,nabla_y_im):
    return np.arctan2(nabla_y_im,nabla_x_im)

img_angle = grad_direction(img_conv_px,img_conv_py)

In [ ]:
# 🛑 Affichage de l'intensité et de l'orientation du gradient
plt.figure(figsize=(15,8))
plt.subplot(131)
plt.imshow(img,cmap='gray')
plt.subplot(132)
plt.imshow(img_grad, cmap='viridis')
plt.title('Intensité du gradient')
plt.subplot(133)
plt.imshow(img_angle, cmap='hsv', vmin=-np.pi, vmax=np.pi)  # Utilisation d'une colormap circulaire
plt.title('Orientation du gradient')
plt.show()

À noter que pour l'orientation du gradient, chaque couleur représente un angle entre $-\pi$ et $\pi$.

In [ ]:
# 🛑 Correspondance entre la couleur et l'angle du gradient
xx = np.linspace(-7,7,15)
X,Y = np.meshgrid(xx,xx)
Z = np.arctan2(Y,X)
plt.imshow(Z, cmap="hsv")
plt.show()

## Votre propre fonction de convolution

### 🛠️ 🚧 👷  À vous de jouer !

Recodez vous-même l'opération de convolution 2D entre une image donnée et un noyau de convolution donné (de taille $(2P+1)\times(2P+1)$), et vérifiez que vous retrouver bien les mêmes résultats que la fonction `conv2d` définie ci-dessus.

⚠️ Pour une convolution 1D ($x \ast y$), le signal glissant $y$ subit un "flip" : ce n'est pas $y$ qui glisse ($\tau \mapsto y(t-\tau)$) mais sa version retournée par rapport à l'axe des ordonnées $y^-$ ($\tau \mapsto y^-(t - \tau) = y(\tau -t)$). En 2D, c'est pareil ! Mais
* ça n'apparaissait pas dans l'animation ci dessus, car le noyau était symétrique par rapport à son pixel central (équivalent d'une fonction paire...)
* c'est géré en interne par les fonctions built-in (comme `sp.signal.convolve`, `sp.signal.convolve2d`, etc)

En revanche, vous <b>devez</b> évidemment appliquer ce "flip" à votre noyau dans votre fonction. Pour cela, vous pouvez appliquer deux fois [`np.flip`](https://numpy.org/doc/stable/reference/generated/numpy.flip.html), une fois avec l'option `axis=0` et une autre fois avec l'option `axis=1`.

In [ ]:
def check_kernel_size(kernel):
    # Vérifie que le noyau donné est de taille (2P+1)x(2P+1)
    try:
        if kernel.ndim != 2:
            raise ValueError("Le noyau doit être en 2D.")
        if kernel.shape[0] % 2 != 1 or kernel.shape[1] % 2 != 1:
            raise ValueError("Les dimensions du noyau doivent être impaires (2P+1)x(2P+1).")
        if kernel.shape[0] != kernel.shape[1]:
            raise ValueError("Les dimensions du noyau doivent être les mêmes.")
    except ValueError as e:
        print(e)
        return False
    return True

def img_padding(img,P):
    # Rajoute P lignes et colonnes avant/après l'image en entrée
    row,col = img.shape
    img_pad = np.zeros((row+2*P,col+2*P))
    img_pad[P:-P,P:-P] = img
    return img_pad

def my_conv(img,kernel):
    ??? # FIXME
    

In [ ]:
np.allclose(my_conv(img,K1),conv2d(img,K1))

In [ ]:
np.allclose(my_conv(img,Px),conv2d(img,Px))

# Bravo !  🎉🥳🍻
Bon boulot ! Vous en avez fini avec ce TP !